# National Health and Nutrition Examination Survey (NHANES)
[https://wwwn.cdc.gov/nchs/nhanes/](https://wwwn.cdc.gov/nchs/nhanes/)

Este laboratório usará dados extraídos do NHANES.

## NHANES datasets from 2013-2014

Foram extraídos dados do [NHANES 2013-2014](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013).

Os dados estão disponíveis em arquivos CSV no diretório `/data/nhanes`. Lá está disponível a documentação do esquema dos arquivos.

# Tarefa 1 - Esquema Relacional

Analise os arquivos CSV que estão no diretório `/data/nhanes` e escreva um esquema relacional referente às tabelas, suas chaves primárias e estrangeiras (não é SQL ainda). Coloque a chave primária com dois underlines antes e dois depois:

\__teste\__

## Resposta Tarefa 1:

 - Pessoa(\__id\__, gender, age)
 - Medicamento(\__code__\, name)
 - Uso_do_Medicamento(\__person_id__\, \__drug_code__\, days_use)
     - __person_id__ CHE => Pessoa
     - __drug_code__ CHE => Medicamento

## Criação das Tabelas em SQL e Importação de Dados

Considere o seguinte exemplo em que é criada uma tabela em SQL e importado dados do arquivo CSV `demographic-person.csv`.

In [9]:
%defaultDatasource jdbc:h2:mem:db

In [10]:
DROP TABLE IF EXISTS Pessoa;
DROP TABLE IF EXISTS Medicamento;
DROP TABLE IF EXISTS Uso_do_Medicamento;

In [11]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../data/demographic-person.csv');

In [12]:
SELECT * FROM Pessoa;

# Tarefa 2 - Criação das Tabelas em SQL e Importação de Dados

Escreva instruções SQL para montar as duas tabelas restante, conforme definido no esquema relacional da **Tarefa 1**.

**Importante:** não defina chave primária para a tabela de uso de medicamento.

Durante a criação das tabelas, realize a importação dos dados do arquivo CSV nas tabelas criadas, conforme exemplo anterior.

In [17]:
CREATE TABLE Medicamento (
  Code VARCHAR(100) NOT NULL,
  Name VARCHAR(100),
  PRIMARY KEY(Code)
) AS SELECT
    code,
    name
FROM CSVREAD('../data/medications-drug.csv');

In [19]:
SELECT * FROM Medicamento;

In [22]:
CREATE TABLE Uso_do_Medicamento (
  PersonId VARCHAR(5) NOT NULL,
  DrugCode VARCHAR(100) NOT NULL,
  DaysUse INTEGER,
  FOREIGN KEY(PersonId)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(DrugCode)
    REFERENCES Medicamento(Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../data/medications-use.csv');

In [23]:
SELECT * FROM Uso_do_Medicamento;

# Tarefa 3 - Consultas Simples

Execute as consultas a seguir.

a) Liste pessoas com idade acima de 60

In [25]:
SELECT P.Id, P.age
FROM Pessoa P WHERE P.age > 60

b) Liste as idades das pessoas cadastradas (sem repetições)

In [26]:
SELECT DISTINCT P.age
    FROM Pessoa P

c) Liste código das pessoas, idade, código dos medicamentos que usaram e dias de uso

In [27]:
SELECT P.Id, P.age, U.DrugCode, U.DaysUse
    FROM Pessoa P, Uso_do_Medicamento U
    WHERE P.Id = U.PersonId

d) Liste código das Pessoas, idade, nome dos medicamentos que usaram e quantos dias de uso

In [28]:
SELECT P.Id, P.age, M.Name, U.DaysUse
    FROM Pessoa P, Uso_do_Medicamento U, Medicamento M
    WHERE P.Id = U.PersonId AND M.Code = U.DrugCode

e) Liste nome dos medicamentos que foram usados por pessoas (sem repetições)

In [36]:
SELECT DISTINCT M.Name
FROM Medicamento M, Uso_do_Medicamento U
WHERE M.Code = U.DrugCode

f) Liste nome dos medicamentos usados por mais de 2.000 dias (sem repetições)

In [35]:
SELECT DISTINCT M.Name
    FROM Medicamento M, Uso_do_Medicamento U
    WHERE M.Code = U.DrugCode AND U.DaysUse > 2000